In [1]:
import torch
import numpy as np
from torchsummaryX import summary
import sklearn
import gc
import zipfile
import pandas as pd
from tqdm.auto import tqdm
import os
import datetime
import wandb
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Device: ',device)

/home/rslim/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Device:  cuda


In [2]:
### PHONEME LIST
PHONEMES = [
            '[SIL]',   'AA',    'AE',    'AH',    'AO',    'AW',    'AY',
            'B',     'CH',    'D',     'DH',    'EH',    'ER',    'EY',
            'F',     'G',     'HH',    'IH',    'IY',    'JH',    'K',
            'L',     'M',     'N',     'NG',    'OW',    'OY',    'P',
            'R',     'S',     'SH',    'T',     'TH',    'UH',    'UW',
            'V',     'W',     'Y',     'Z',     'ZH',    '[SOS]', '[EOS]']

In [3]:
# uncomment this if you are running directly in kaggle notebook
# !pip install --upgrade --force-reinstall --no-deps kaggle==1.5.8
# !mkdir /root/.kaggle

# with open("/root/.kaggle/kaggle.json", "w+") as f:
#     f.write('{"username":"Replace this with your Kaggle Username","key":"Replace this with your kaggle API key"}')
#     # Put your kaggle username & key here

# !chmod 600 /root/.kaggle/kaggle.json

In [4]:
if not os.path.exists("11785-hw1p2-f24" + ".zip"):
    !kaggle competitions download -c 11785-hw1p2-f24

if not os.path.exists("11785-f24-hw1p2"):
    !unzip -qo 11785-hw1p2-f24.zip

In [5]:
"""
Implements a scanning MLP 
"""

'\nImplements a scanning MLP \n'

In [6]:
# # Dataset class to load train and validation data
# root = os.path.dirname(os.path.abspath("__file__"))

# context = 20
# # phonemes = PHONEMES
# partition="train-clean-100"

# dataset_dir = "11785-f24-hw1p2"
# dataset_path = os.path.join(root, dataset_dir)

# mfcc_dir = os.path.join(dataset_path, partition, 'mfcc')
# transcript_dir = os.path.join(dataset_path, partition, 'transcript')

# # list files in self.mfcc_dir using os.listdir in sorted order
# mfcc_names = []
# files = os.listdir(mfcc_dir)
# for file in files:
#     mfcc_names.append(os.path.join(mfcc_dir, file))

# transcript_names = []
# files = os.listdir(transcript_dir)
# for file in files:
#     transcript_names.append(os.path.join(transcript_dir, file))

# mfcc_names = sorted(mfcc_names)
# transcript_names = sorted(transcript_names)

# # Making sure that we have the same no. of mfcc and transcripts
# assert len(mfcc_names) == len(transcript_names)

# mfccs, transcripts = [], []

# # Iterate through mfccs and transcripts
# for i in range(len(mfcc_names)):
#     # Load a single mfcc
#     mfcc = np.load(mfcc_names[i])
    
#     # Do cepstral normalization of mfcc
#     mfcc_mean = np.mean(mfcc, axis=0)
#     mfcc_std = np.std(mfcc, axis=0)
#     mfcc = (mfcc - mfcc_mean) / mfcc_std
    
#     transcript = np.load(transcript_names[i])[1:-1]  # remove [SOS] at beginning, and [EOS] at end efficiently.
    
#     mfccs.append(mfcc)
#     transcripts.append(transcript)



# mfccs = np.concatenate(mfccs, axis=0)
# transcripts = np.concatenate(transcripts, axis=0)

# mfccs = np.pad(mfccs, ((context, context), (0, 0)), mode='constant')
# # The available phonemes in the transcript are of string data type
# # But the neural network cannot predict strings as such.
# # Hence, we map these phonemes to integers

# # Map the phonemes to their corresponding list indexes in self.phonemes
# phoneme_to_index = {phoneme: idx for idx, phoneme in enumerate(PHONEMES)}
# transcripts = np.array([phoneme_to_index[phoneme] for phoneme in transcripts])

In [7]:
# mfccs.shape

In [8]:
# transcripts.shape

In [8]:
# np.load(mfcc_names[5]).shape

In [9]:
# np.load(transcript_names[5]).shape

In [10]:
# mfccs[1:1+2*20+1].shape

In [11]:
# mfccs[1:1+2*20+1].flatten().shape

In [9]:
# ind = 0
# frames = mfccs[ind:ind + 2 * context + 1]
# frames = frames.flatten()
# print(frames.shape)
# print(frames)
# phonemes = transcripts[0]
# print(phonemes.shape)
# print(phonemes)


In [10]:
# len(PHONEMES)

In [11]:
# mfccs.shape

In [12]:
# Dataset class to load train and validation data
root = os.path.dirname(os.path.abspath("__file__"))

class AudioDataset(torch.utils.data.Dataset):

    def __init__(self, root, phonemes=PHONEMES, context=0, partition="train-clean-100"):  # Feel free to add more arguments

        self.context = context
        self.phonemes = phonemes

        dataset_dir = "11785-f24-hw1p2"
        dataset_path = os.path.join(root, dataset_dir)
        
        self.mfcc_dir = os.path.join(dataset_path, partition, 'mfcc')
        self.transcript_dir = os.path.join(dataset_path, partition, 'transcript')
        
        # list files in self.mfcc_dir using os.listdir in sorted order
        mfcc_names = []
        files = os.listdir(self.mfcc_dir)
        for file in files:
            mfcc_names.append(os.path.join(self.mfcc_dir, file))

        transcript_names = []
        files = os.listdir(self.transcript_dir)
        for file in files:
            transcript_names.append(os.path.join(self.transcript_dir, file))

        mfcc_names = sorted(mfcc_names)
        transcript_names = sorted(transcript_names)
        
        # Making sure that we have the same no. of mfcc and transcripts
        assert len(mfcc_names) == len(transcript_names)

        self.mfccs, self.transcripts = [], []

        # Iterate through mfccs and transcripts
        for i in range(len(mfcc_names)):
            # Load a single mfcc
            mfcc = np.load(mfcc_names[i])
            
            # Do cepstral normalization of mfcc
            mfcc_mean = np.mean(mfcc, axis=0)
            mfcc_std = np.std(mfcc, axis=0)
            mfcc = (mfcc - mfcc_mean) / mfcc_std
            
            transcript = np.load(transcript_names[i])[1:-1]  # remove [SOS] at beginning, and [EOS] at end efficiently.
            
            self.mfccs.append(mfcc)
            self.transcripts.append(transcript)

        

        self.mfccs = np.concatenate(self.mfccs, axis=0)
        self.transcripts = np.concatenate(self.transcripts, axis=0)

        assert len(self.mfccs) == len(self.transcripts)
        
        # Length of the dataset is now the length of concatenated mfccs/transcripts
        self.length = len(self.mfccs)

        # self.mfcc is an array of the format (Frames x Features).
        # Our goal is to recognize phonemes of each frame
        # We can introduce context by padding zeros on top and bottom of self.mfcc
        self.mfccs = np.pad(self.mfccs, ((self.context, self.context), (0, 0)), mode='constant')
        # The available phonemes in the transcript are of string data type
        # But the neural network cannot predict strings as such.
        # Hence, we map these phonemes to integers
        
        # Map the phonemes to their corresponding list indexes in self.phonemes
        phoneme_to_index = {phoneme: idx for idx, phoneme in enumerate(self.phonemes)}
        self.transcripts = np.array([phoneme_to_index[phoneme] for phoneme in self.transcripts])
        # Now, if an element in self.transcript is 0, it means that it is 'SIL' (as per the above example)

        # print("self.mfccs.shape ", self.mfccs.shape)
        # print("self.transcripts.shape ", self.transcripts.shape)

    def __len__(self):
        return self.length

    def __getitem__(self, ind):
        frames = self.mfccs[ind:ind + 2 * self.context + 1]
        frames = frames.flatten()
        frames = torch.FloatTensor(frames)  # Convert to tensors
        phonemes = torch.tensor(self.transcripts[ind])
        return frames, phonemes

# train_dataset = AudioDataset(root, context=0)
# x, y = train_dataset[len(train_dataset)-1]
# print(x)
# print(y)

In [13]:
class AudioTestDataset(torch.utils.data.Dataset):
    
    def __init__(self, root, phonemes=PHONEMES, context=0, partition="test-clean"):

        self.context = context
        self.phonemes = phonemes
        
        dataset_dir = "11785-f24-hw1p2"
        dataset_path = os.path.join(root, dataset_dir)

        self.mfcc_dir = os.path.join(dataset_path, partition, 'mfcc')

        # list files in self.mfcc_dir using os.listdir in sorted order
        mfcc_names = []
        files = os.listdir(self.mfcc_dir)
        for file in files:
            mfcc_names.append(os.path.join(self.mfcc_dir, file))

        mfcc_names = sorted(mfcc_names)

        self.mfccs = []
        
        # Iterate through mfccs and transcripts
        for i in range(len(mfcc_names)):
            # Load a single mfcc
            mfcc = np.load(mfcc_names[i])
            
            # Do cepstral normalization of mfcc
            mfcc_mean = np.mean(mfcc, axis=0)
            mfcc_std = np.std(mfcc, axis=0)
            mfcc = (mfcc - mfcc_mean) / mfcc_std
            
            self.mfccs.append(mfcc)

        self.mfccs = np.concatenate(self.mfccs, axis=0)

        self.length = len(self.mfccs)

        self.mfccs = np.pad(self.mfccs, ((self.context, self.context), (0, 0)), mode='constant')

    def __len__(self):
        return self.length

    def __getitem__(self, ind):
        frames = self.mfccs[ind:ind + 2 * self.context + 1]
        frames = frames.flatten()
        frames = torch.FloatTensor(frames)  # Convert to tensors
        return frames

# test_dataset = AudioTestDataset(root)
# x = test_dataset[len(test_dataset)-1]
# print(x)


In [14]:
config = {
    'epochs' : 5,
    'batch_size' : 1024,
    'context' : 20,
    'init_lr' : 1e-3,
    'architecture' : 'very-low-cutoff'
}

In [15]:
train_data = AudioDataset(root, context=config['context'], partition='train-clean-100')

val_data = AudioDataset(root, context=config['context'], partition='dev-clean')

test_data = AudioTestDataset(root, context=config['context'], partition='test-clean')

In [16]:
# x, y = train_data[len(train_data)-12]

In [17]:
# x.shape

In [18]:
# y

In [19]:
# Define dataloaders for train, val and test datasets
# Dataloaders will yield a batch of frames and phonemes of given batch_size at every iteration
# We shuffle train dataloader but not val & test dataloader. Why?

train_loader = torch.utils.data.DataLoader(
    dataset     = train_data,
    num_workers = 4,
    batch_size  = config['batch_size'],
    pin_memory  = True,
    shuffle     = True
)

val_loader = torch.utils.data.DataLoader(
    dataset     = val_data,
    num_workers = 2,
    batch_size  = config['batch_size'],
    pin_memory  = True,
    shuffle     = False
)

test_loader = torch.utils.data.DataLoader(
    dataset     = test_data,
    num_workers = 2,
    batch_size  = config['batch_size'],
    pin_memory  = True,
    shuffle     = False
)


print("Batch size     : ", config['batch_size'])
print("Context        : ", config['context'])
print("Input size     : ", (2 * config['context'] + 1) * 28)
print("Output symbols : ", len(PHONEMES))

print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
print("Validation dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))
print("Test dataset samples = {}, batches = {}".format(test_data.__len__(), len(test_loader)))

Batch size     :  1024
Context        :  20
Input size     :  1148
Output symbols :  42
Train dataset samples = 36091157, batches = 35246
Validation dataset samples = 1928204, batches = 1884
Test dataset samples = 1934138, batches = 1889


In [20]:
# (2 * config['context'] + 1) * 28

In [21]:
# Testing code to check if your data loaders are working
for i, data in enumerate(train_loader):
    frames, phoneme = data
    print(frames.shape, phoneme.shape)
    break

torch.Size([1024, 1148]) torch.Size([1024])


In [22]:
# This architecture will make you cross the very low cutoff
# However, you need to run a lot of experiments to cross the medium or high cutoff
class Network(torch.nn.Module):

    def __init__(self, input_size, output_size):

        super(Network, self).__init__()

        self.model = torch.nn.Sequential(
            torch.nn.Linear(input_size, 512),
            torch.nn.ReLU(),
            torch.nn.Linear(512, output_size)
        )

    def forward(self, x):
        out = self.model(x)
        return out

In [23]:
INPUT_SIZE = (2 * config['context'] + 1) * 28
model = Network(INPUT_SIZE, len(train_data.phonemes)).to(device)
summary(model, frames.to(device))

----------------------------------------------------------------------------------------------------
Layer                   Kernel Shape         Output Shape         # Params (K)      # Mult-Adds (M)
0_Linear                 [1148, 512]          [1024, 512]               588.29                 0.59
1_ReLU                             -          [1024, 512]                    -                    -
2_Linear                   [512, 42]           [1024, 42]                21.55                 0.02
# Params:    609.83K
# Mult-Adds: 0.61M
----------------------------------------------------------------------------------------------------


In [24]:
criterion = torch.nn.CrossEntropyLoss()  # Define loss function.
# We use CE for multi-class classification

optimizer = torch.optim.Adam(model.parameters(), lr=config['init_lr'])  # Defining optimizer
# Recommended : Define scheduler for Learning Rate, 
# including but not limited to StepLR, MultiStepLR, CosineAnnealingLR, ReduceLROnPlateau, etc.
# You can refer to Pytorch documentation for more information on how to use them.

# Is your training time very high?
# Look into mixed precision training if your GPU (Tesla T4, V100, etc) can make use of it
# Refer - https://pytorch.org/docs/stable/notes/amp_examples.html

In [25]:
torch.cuda.empty_cache()
gc.collect()

15

In [ ]:
def train(model, dataloader, optimizer, criterion):

    model.train()
    tloss, tacc = 0, 0  # Monitoring loss and accuracy
    batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train')

    for i, (frames, phonemes) in enumerate(dataloader):
        
        # Initialize Gradients
        optimizer.zero_grad()

        # Move data to device (ideally GPU)
        frames = frames.to(device)
        phonemes = phonemes.to(device)

        # forward propagation
        logits = model(frames)

        # loss calculation
        loss = criterion(logits, phonemes)

        # back propagation
        loss.backward()

        # gradient descent
        optimizer.step()

        tloss += loss.item()
        # print("logits", logits)
        # print("logits.shape", logits.shape)
        # print("torch.argmax(logits, dim=1)", torch.argmax(logits, dim=1))
        # print("torch.argmax(logits, dim=1).shape", torch.argmax(logits, dim=1).shape)
        # print("torch.argmax(logits, dim=1)==phonemes", torch.argmax(logits, dim=1)==phonemes)
        tacc += torch.sum(torch.argmax(logits, dim=1) == phonemes).item() / logits.shape[0]

        batch_bar.set_postfix(loss="{:.04f}".format(float(tloss / (i + 1))),
                              acc="{:.04f}%".format(float(tacc*100 / (i + 1))))
        batch_bar.update()

        # release memory
        del frames, phonemes, logits
        torch.cuda.empty_cache()

    batch_bar.close()
    tloss /= len(dataloader)
    tacc /= len(dataloader)

    return tloss, tacc

In [ ]:
def eval(model, dataloader):

    model.eval()  # set model in evaluation mode
    vloss, vacc = 0, 0  # monitoring loss and accuracy
    batch_bar = tqdm(total=len(val_loader), dynamic_ncols=True, position=0, leave=False, desc='Val')

    for i, (frames, phonemes) in enumerate(dataloader):

        # move data to device (ideally a GPU)
        frames = frames.to(device)
        phonemes = phonemes.to(device)

        # makes sure that there are no gradients computed as we are not training the model now
        with torch.inference_mode():
            # forward propagation
            logits = model(frames)

            # loss calculation
            loss = criterion(logits, phonemes)

        vloss += loss.item()
        vacc += torch.sum(torch.argmax(logits, dim=1) == phonemes).item() / logits.shape[0]

        # Do you think we need loss.backward() and optimizer.step() here?

        batch_bar.set_postfix(loss="{:.04f}".format(float(vloss / (i + 1))),
                              acc="{:.04f}%".format(float(vacc*100 / (i + 1))))

        # Release memory
        del frames, phonemes, logits
        torch.cuda.empty_cache()

    batch_bar.close()
    vloss /= len(dataloader)
    vacc /= len(dataloader)

    return vloss, vacc

In [28]:
wandb.login(key="c62dd63a448dbe45c79277111818735bed8f5244") #API Key is in your wandb account, under settings (wandb.ai/settings)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/rslim/.netrc
wandb: Currently logged in as: reinaldymaslim (reinaldymaslim-nanyang-technological-university-singapore) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [29]:
# Create your wandb run
run = wandb.init(
    name    = "first-run", ### Wandb creates random run names if you skip this field, we recommend you give useful names
    reinit  = True, ### Allows reinitalizing runs when you re-run this cell
    #id     = "y28t31uz", ### Insert specific run id here if you want to resume a previous run
    #resume = "must", ### You need this to resume previous runs, but comment out reinit = True when using this
    project = "hw1p2", ### Project should be created in your wandb account
    config  = config ### Wandb Config for your run
)

wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


In [30]:
### Save your model architecture as a string with str(model)
model_arch  = str(model)

### Save it in a txt file
arch_file   = open("model_arch.txt", "w")
file_write  = arch_file.write(model_arch)
arch_file.close()

### log it in your wandb run with wandb.save()
wandb.save('model_arch.txt')

['/home/rslim/proj/dl/dl2/HW1/P2/wandb/run-20250729_141202-x292tgk6/files/model_arch.txt']

In [31]:
# train(model, train_loader, optimizer, criterion)

In [33]:
# Iterate over number of epochs to train and evaluate your model
torch.cuda.empty_cache()
gc.collect()
wandb.watch(model, log="all")

for epoch in range(config['epochs']):

    print("\nEpoch {}/{}".format(epoch+1, config['epochs']))

    curr_lr                 = float(optimizer.param_groups[0]['lr'])
    train_loss, train_acc   = train(model, train_loader, optimizer, criterion)
    val_loss, val_acc       = eval(model, val_loader)

    print("\tTrain Acc {:.04f}%\tTrain Loss {:.04f}\t Learning Rate {:.07f}".format(train_acc*100, train_loss, curr_lr))
    print("\tVal Acc {:.04f}%\tVal Loss {:.04f}".format(val_acc*100, val_loss))

    ### Log metrics at each epoch in your run
    # Optionally, you can log at each batch inside train/eval functions
    # (explore wandb documentation/wandb recitation)
    wandb.log({'train_acc': train_acc*100, 'train_loss': train_loss,
               'val_acc': val_acc*100, 'valid_loss': val_loss, 'lr': curr_lr})

    ### Highly Recommended: Save checkpoint in drive and/or wandb if accuracy is better than your current best

### Finish your wandb run
# run.finish()


Epoch 1/5


	Train Acc 67.9945%	Train Loss 1.0404	 Learning Rate 0.0010000
	Val Acc 67.9164%	Val Loss 1.0387

Epoch 2/5


	Train Acc 69.7877%	Train Loss 0.9782	 Learning Rate 0.0010000
	Val Acc 68.6853%	Val Loss 1.0093

Epoch 3/5


	Train Acc 70.4589%	Train Loss 0.9549	 Learning Rate 0.0010000
	Val Acc 69.5162%	Val Loss 0.9861

Epoch 4/5


	Train Acc 70.8440%	Train Loss 0.9415	 Learning Rate 0.0010000
	Val Acc 69.5022%	Val Loss 0.9832

Epoch 5/5


	Train Acc 71.0926%	Train Loss 0.9329	 Learning Rate 0.0010000
	Val Acc 69.8510%	Val Loss 0.9722


In [36]:
def test(model, test_loader):
    ### What you call for model to perform inference?
    model.eval() # TODO train or eval?

    ### List to store predicted phonemes of test data
    test_predictions = []

    ### Which mode do you need to avoid gradients?
    with torch.inference_mode(): # TODO

        for i, mfccs in enumerate(tqdm(test_loader)):
 
            mfccs   = mfccs.to(device)

            logits  = model(mfccs)

            # print(logits)

            ### Get most likely predicted phoneme with argmax
            predicted_phonemes_indices = torch.argmax(logits, dim=1)
            # print(predicted_phonemes)
            # print(predicted_phonemes.shape)
            predicted_phonemes = [PHONEMES[i] for i in predicted_phonemes_indices.cpu().numpy()]
            
            ### How do you store predicted_phonemes with test_predictions? Hint, look at eval
            test_predictions.extend(predicted_phonemes)

    return test_predictions

In [37]:
predictions = test(model, test_loader)


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1889/1889 [00:04<00:00, 452.74it/s]


In [39]:
### Create CSV file with predictions
with open("./submission.csv", "w+") as f:
    f.write("id,label\n")
    for i in range(len(predictions)):
        f.write("{},{}\n".format(i, predictions[i]))

In [17]:
# root = os.path.dirname(os.path.abspath("__file__"))

In [34]:
# # Dataset class to load train and validation data
# dataset_dir = "11785-f24-hw1p2"
# root = os.path.dirname(os.path.abspath("__file__"))
# dataset_path = os.path.join(root, dataset_dir)
# train_and_test = ["train-clean-100"]

In [35]:
# mfcc_dir = [os.path.join(dataset_path, folder, 'mfcc') for folder in train_and_test]
# transcript_dir = [os.path.join(dataset_path, folder, 'transcript') for folder in train_and_test]

In [36]:
# mfcc_names = []
# for path in mfcc_dir:
#     files = os.listdir(path)
#     for file in files:
#         mfcc_names.append(os.path.join(path, file))
# mfcc_names = sorted(mfcc_names)

In [37]:
# transcript_names = []
# for path in transcript_dir:
#     files = os.listdir(path)
#     for file in files:
#         transcript_names.append(os.path.join(path, file))
# transcript_names = sorted(transcript_names)

In [35]:
# mfccs = []
# transcripts = []
# assert len(mfcc_names) == len(transcript_names)
# for i in range(len(mfcc_names)):
#     mfccs.append(np.load(mfcc_names[i]))
#     transcripts.append(np.load(transcript_names[i])[1:-1])
# print(len(mfccs))
# print(len(transcripts))

In [49]:
# phoneme_to_index = {phoneme: idx for idx, phoneme in enumerate(PHONEMES)}

In [33]:
# phoneme_to_index

In [51]:
# transcripts = np.array([phoneme_to_index[phoneme] for phoneme in transcripts])

In [34]:
# np.where(transcripts==12)[0].shape

In [41]:
### Finish your wandb run
run.finish()

lr,▁▁▁▁▁
train_acc,▁▅▇▇█
train_loss,█▄▂▂▁
val_acc,▁▄▇▇█
valid_loss,█▅▂▂▁
lr,0.001
train_acc,71.0926
train_loss,0.93287
val_acc,69.85102
valid_loss,0.97223


In [40]:
!kaggle competitions submit -c 11785-hw1p2-f24 -f ./submission.csv -m "try submit"

100%|██████████████████████████████████████| 19.4M/19.4M [00:03<00:00, 5.91MB/s]
Successfully submitted to 11785 HW1P2 Fall 2024